In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
from torch import cuda 

device = 'cuda' if cuda.is_available() else 'cpu'
cuda.empty_cache()
print(device)

## Custom Data
### Create mask images

In [4]:
import json
from glob import glob
from utils import poly2mask, discrete_cmap, label2id, id2label
import matplotlib.pyplot as plt 

### RENAME DIR NAMES IN ENGLISH

In [5]:
img_dir = "nia23soc_local/Original/underground/underground_tunnel/"
label_dir = "nia23soc_local/Labeling/underground/underground_tunnel/"
label_list = glob(label_dir+"/*.json")

### Generate mask maps

In [12]:
if True:
    save_dir = "./nia23soc_local/mask_overlay/" # Check image and mask orientation
    labels = []
    for fn_label in label_list:
        fn_img = fn_label.replace("Labeling", "Original").replace(".json", ".jpg")
        poly2mask(fn_label, fn_img, save_dir=save_dir)

In [13]:
mask_list = glob("./nia23soc_local/mask/*.png")
print(len(mask_list))

20732


#### Create Dataset

In [14]:
from datasets import Dataset, DatasetDict, Image
from sklearn.model_selection import train_test_split

def cast_dataset(image, label):
    dataset = Dataset.from_dict({"pixel_values": sorted(image),
                                 "label": sorted(label)})
    dataset = dataset.cast_column("pixel_values", Image())
    dataset = dataset.cast_column("label", Image())
    return dataset

def create_dataset(image_paths, label_paths, test_size=0.2, valid_size=0.1, random_state=123):
    img_train, img_rem, label_train, label_rem = train_test_split(image_paths, 
                                                                    label_paths, 
                                                                    test_size=test_size+valid_size, 
                                                                    random_state=random_state)
    
    img_valid, img_test, label_valid, label_test = train_test_split(img_rem, 
                                                                    label_rem, 
                                                                    test_size = 1 - valid_size/test_size, 
                                                                    random_state=random_state)
    
    train_dataset = cast_dataset(img_train, label_train)
    test_dataset = cast_dataset(img_test, label_test)
    valid_dataset = cast_dataset(img_valid, label_valid)

    return train_dataset, test_dataset, valid_dataset


In [15]:
img_list = glob(img_dir+"/*.jpg")
# step 1: create Dataset objects
train_dataset, test_dataset, validation_dataset = create_dataset(img_list, mask_list, test_size=0.2, valid_size=0.1, random_state=123)
#validation_dataset = create_dataset(image_paths_validation, label_paths_validation)

# step 2: create DatasetDict
dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset, # optional??
    "validation": validation_dataset,
  }
)



#### Meta data needed

In [16]:
# Segformer expects 'id2label.json' file
with open('id2label.json', 'w') as fp:
    json.dump(id2label, fp)

In [17]:
from torchvision.transforms import ColorJitter
from transformers import SegformerFeatureExtractor, SegformerImageProcessor

#feature_extractor = SegformerFeatureExtractor()
feature_extractor = SegformerImageProcessor(do_reduce_labels=True)
jitter = ColorJitter(brightness=0.25, contrast=0.25, saturation=0.25, hue=0.1) 

def train_transforms(example_batch):
    images = [jitter(x) for x in example_batch['pixel_values']]
    labels = [x for x in example_batch['label']]
    inputs = feature_extractor(images, labels)
    return inputs


def val_transforms(example_batch):
    images = [x for x in example_batch['pixel_values']]
    labels = [x for x in example_batch['label']]
    inputs = feature_extractor(images, labels)
    return inputs



### Fine tuning (never tried from scratch...)

In [18]:
from transformers import SegformerForSemanticSegmentation

pretrained_model_name = "nvidia/mit-b0" 
model = SegformerForSemanticSegmentation.from_pretrained(
    pretrained_model_name,
    id2label=id2label,
    label2id=label2id
)
model = model.to(device)

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head.classifier.bias', 'decode_head.linear_fuse.weight', 'decode_head.linear_c.2.proj.weight', 'decode_head.batch_norm.weight', 'decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.2.proj.bias', 'decode_head.classifier.weight', 'decode_head.batch_norm.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.batch_norm.running_mean', 'decode_head.linear_c.3.proj.weight', 'decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_c.3.proj.bias', 'decode_head.linear_c.1.proj.weight', 'decode_head.batch_norm.running_var']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Trainer

In [19]:
from transformers import TrainingArguments

epochs = 150
lr = 0.00006
batch_size = 16

# Refer to https://huggingface.co/docs/transformers/v4.17.0/en/main_classes/trainer#transformers.TrainingArguments 
# for more details on the Trainer arguments
training_args = TrainingArguments(
    "segformer-b0-finetuned-segments-sidewalk-outputs",
    learning_rate=lr,
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    dataloader_num_workers=6,
    save_total_limit=3,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_steps=20,
    eval_steps=20,
    logging_steps=1,
    eval_accumulation_steps=5,
    load_best_model_at_end=True,
    push_to_hub=False,
    resume_from_checkpoint=True,
    #hub_model_id=hub_model_id,
    #hub_strategy="end",
)

#### Evaluation metric (mIOU)

In [20]:
import torch
from torch import nn
import evaluate

metric = evaluate.load("mean_iou")

def compute_metrics(eval_pred):
  
  with torch.no_grad():
    logits, labels = eval_pred
    print("LABELS", np.unique(labels))
    logits_tensor = torch.from_numpy(logits)
    # scale the logits to the size of the label
    logits_tensor = nn.functional.interpolate(
        logits_tensor,
        size=labels.shape[-2:],
        mode="bilinear",
        align_corners=False,
    ).argmax(dim=1)

    pred_labels = logits_tensor.detach().cpu().numpy()
    print("PRED LABELS", np.unique(pred_labels))
    # currently using _compute instead of compute
    # see this issue for more info: https://github.com/huggingface/evaluate/pull/328#issuecomment-1286866576
    metrics = metric._compute(
            predictions=pred_labels,
            references=labels,
            num_labels=len(id2label),
            ignore_index=0,
            reduce_labels=feature_extractor.do_reduce_labels,
        )
    
    # add per category metrics as individual key-value pairs
    per_category_accuracy = metrics.pop("per_category_accuracy").tolist()
    per_category_iou = metrics.pop("per_category_iou").tolist()

    metrics.update({f"accuracy_{id2label[i+1]}": v for i, v in enumerate(per_category_accuracy)})
    metrics.update({f"iou_{id2label[i+1]}": v for i, v in enumerate(per_category_iou)})
    
    return metrics

#### Instantiate a Trainer

In [21]:
from transformers import Trainer

train_dataset.set_transform(train_transforms)
test_dataset.set_transform(val_transforms)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

In [22]:
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hoseung (dinsight). Use `wandb login --relogin` to force relogin


  0%|          | 0/136050 [00:00<?, ?it/s]

{'loss': 2.2442, 'learning_rate': 5.999955898566704e-05, 'epoch': 0.0}
{'loss': 2.0989, 'learning_rate': 5.999911797133407e-05, 'epoch': 0.0}
{'loss': 2.2423, 'learning_rate': 5.999867695700111e-05, 'epoch': 0.0}
{'loss': 2.3693, 'learning_rate': 5.999823594266814e-05, 'epoch': 0.0}
{'loss': 2.0343, 'learning_rate': 5.9997794928335175e-05, 'epoch': 0.01}
{'loss': 2.359, 'learning_rate': 5.9997353914002205e-05, 'epoch': 0.01}
{'loss': 1.6553, 'learning_rate': 5.999691289966924e-05, 'epoch': 0.01}
{'loss': 1.9049, 'learning_rate': 5.999647188533627e-05, 'epoch': 0.01}
{'loss': 2.3995, 'learning_rate': 5.999603087100331e-05, 'epoch': 0.01}
{'loss': 2.2614, 'learning_rate': 5.999558985667034e-05, 'epoch': 0.01}
{'loss': 2.4294, 'learning_rate': 5.999514884233738e-05, 'epoch': 0.01}
{'loss': 2.296, 'learning_rate': 5.999470782800441e-05, 'epoch': 0.01}
{'loss': 1.9827, 'learning_rate': 5.9994266813671446e-05, 'epoch': 0.01}
{'loss': 1.7722, 'learning_rate': 5.999382579933848e-05, 'epoch': 0

  0%|          | 0/195 [00:00<?, ?it/s]

LABELS [  0   1   2   3   4   5   6   7   8   9 255]
PRED LABELS [0 1 2 3 4 5 6 7 8 9]


...

/home/hoseung/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/08bc20f4f895f3caf75fb9e3fada1404bded3c3265243d05327cbb3b9326ffe9/mean_iou.py:260: RuntimeWarning: invalid value encountered in divide
  acc = total_area_intersect / total_area_label


{'eval_loss': 2.004974126815796, 'eval_mean_iou': 0.006232395529362599, 'eval_mean_accuracy': 0.08333930276963059, 'eval_overall_accuracy': 0.16257714907011553, 'eval_accuracy_crack': nan, 'eval_accuracy_reticular crack': 0.03976076508315626, 'eval_accuracy_detachment': 0.0008928847415701234, 'eval_accuracy_spalling': 0.03909252092937204, 'eval_accuracy_efflorescene': 0.5869682514029463, 'eval_accuracy_leak': 0.0, 'eval_accuracy_rebar': 0.0, 'eval_accuracy_material separation': 0.0, 'eval_accuracy_exhilaration': 0.0, 'eval_accuracy_damage': nan, 'eval_iou_crack': 0.0, 'eval_iou_reticular crack': 0.00017034860275249547, 'eval_iou_detachment': 0.00011921923209962673, 'eval_iou_spalling': 0.015619511064462018, 'eval_iou_efflorescene': 0.04641487639431184, 'eval_iou_leak': 0.0, 'eval_iou_rebar': 0.0, 'eval_iou_material separation': 0.0, 'eval_iou_exhilaration': 0.0, 'eval_iou_damage': 0.0, 'eval_runtime': 103.8259, 'eval_samples_per_second': 29.954, 'eval_steps_per_second': 1.878, 'epoch':

  0%|          | 0/195 [00:00<?, ?it/s]

LABELS [  0   1   2   3   4   5   6   7   8   9 255]


KeyboardInterrupt: 

In [ ]:
model.save_pretrained("fine_tuend_segformer_b0")